In [2]:
from pandasai.llm import HuggingFaceTextGen
from pandasai import SmartDataframe
import os
import pandas as pd
from pandasai import Agent
from pandasai.connectors import PandasConnector

In [3]:


llm = HuggingFaceTextGen(
    inference_server_url="http://127.0.0.1:8080"
)
#df = SmartDataframe("data.csv", config={"llm": llm})


/home/Ubuntu/projects/pdai/.venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/Ubuntu/projects/pdai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('claim_data.csv')

In [5]:
df.head()

,Unnamed: 0,Dealer Code,Dealer Name,Dealer State,Region,Status,Status Reson,Chassis no,TMInvoiceno,TM Invoice Date,...,Claim ID,G_Claim ID,Scheme ID,Amount,SAP Doc no.,SAP Doc Date,Scheme Type,Scheme Type-Name,Scheme Type Category,LOB
0,0,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT558005PVJ50387,723276640,2023-09-29,...,CVA24848666,0,SCHM12-13493,10000.0,728322645,2024-04-06,Chola Cash,Cash discount in lieu of Chola,Consumer,SCVPass
1,1,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT558005PVJ50387,723276640,2023-09-29,...,CVB24880194,0,SCHM12-13480,10000.0,728322646,2024-04-06,Exchange,Exchange Discount,Consumer,SCVPass
2,2,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT535351PYP56736,723477439,2023-12-29,...,CVD241040310,0,SCHM03-18345,20000.0,Flat,2023-12-29,Exchange,Exchange Discount,Consumer,Pickups
3,3,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT535351RYB04948,728111745,2024-02-29,...,CVD241040342,0,SCHM03-18345,20000.0,Flat,2024-02-29,Exchange,Exchange Discount,Consumer,Pickups
4,4,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT556013PVP68963,728059236,2024-01-31,...,CVD241040436,0,SCHM03-18341,10000.0,Flat,2024-01-31,Exchange,Exchange Discount,Consumer,SCV Cargo


In [6]:
if 'Unnamed: 0' in df.columns:
    # Drop the column if it exists
    df = df.drop(columns=['Unnamed: 0'])

In [7]:
df['TM Invoice Date'] = df['TM Invoice Date'].str.strip()
df['Retail invoice Date'] = df['Retail invoice Date'].str.strip()
df['SAP Doc Date'] = df['SAP Doc Date'].str.strip()


In [8]:
df['TM Invoice Date'] = pd.to_datetime(df['TM Invoice Date'], errors='coerce', infer_datetime_format=True)
df['Retail invoice Date'] = pd.to_datetime(df['Retail invoice Date'], errors='coerce', infer_datetime_format=True)
df['SAP Doc Date'] = pd.to_datetime(df['SAP Doc Date'], errors='coerce', infer_datetime_format=True)

In [9]:
# Convert date columns to datetime format
# df['TM Invoice Date'] = pd.to_datetime(df['TM Invoice Date'], format='%y-%m-%d', errors='coerce')
# df['Retail invoice Date'] = pd.to_datetime(df['Retail invoice Date'], format='%y-%m-%d', errors='coerce')
# df['SAP Doc Date'] = pd.to_datetime(df['SAP Doc Date'], format='%y-%m-%d', errors='coerce')

# Convert 'TMInvoiceno' to string
df['TMInvoiceno'] = df['TMInvoiceno'].astype(str)
df['Dealer Code'] = df['Dealer Code'].astype(str)
df['G_Claim ID'] = df['G_Claim ID'].astype(str)


In [10]:
nat_rows = df[df['TM Invoice Date'].isna() | df['Retail invoice Date'].isna() | df['SAP Doc Date'].isna()]
print(nat_rows[['TM Invoice Date', 'Retail invoice Date', 'SAP Doc Date']])

Empty DataFrame
Columns: [TM Invoice Date, Retail invoice Date, SAP Doc Date]
Index: []


In [11]:
df.head()

,Dealer Code,Dealer Name,Dealer State,Region,Status,Status Reson,Chassis no,TMInvoiceno,TM Invoice Date,Retail invoice no,...,Claim ID,G_Claim ID,Scheme ID,Amount,SAP Doc no.,SAP Doc Date,Scheme Type,Scheme Type-Name,Scheme Type Category,LOB
0,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT558005PVJ50387,723276640,2023-09-29,IAUTGH2324001350,...,CVA24848666,0,SCHM12-13493,10000.0,728322645,2024-04-06,Chola Cash,Cash discount in lieu of Chola,Consumer,SCVPass
1,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT558005PVJ50387,723276640,2023-09-29,IAUTGH2324001350,...,CVB24880194,0,SCHM12-13480,10000.0,728322646,2024-04-06,Exchange,Exchange Discount,Consumer,SCVPass
2,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT535351PYP56736,723477439,2023-12-29,IAUTGH2324002184,...,CVD241040310,0,SCHM03-18345,20000.0,Flat,2023-12-29,Exchange,Exchange Discount,Consumer,Pickups
3,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT535351RYB04948,728111745,2024-02-29,IAUTGH2324002193,...,CVD241040342,0,SCHM03-18345,20000.0,Flat,2024-02-29,Exchange,Exchange Discount,Consumer,Pickups
4,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT556013PVP68963,728059236,2024-01-31,IAUTGH2324002052,...,CVD241040436,0,SCHM03-18341,10000.0,Flat,2024-01-31,Exchange,Exchange Discount,Consumer,SCV Cargo


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5108 entries, 0 to 5107
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Dealer Code           5108 non-null   object        
 1   Dealer Name           5108 non-null   object        
 2   Dealer State          5108 non-null   object        
 3   Region                5108 non-null   object        
 4   Status                5108 non-null   object        
 5   Status Reson          5108 non-null   object        
 6   Chassis no            5108 non-null   object        
 7   TMInvoiceno           5108 non-null   object        
 8   TM Invoice Date       5108 non-null   datetime64[ns]
 9   Retail invoice no     5108 non-null   object        
 10  Retail invoice Date   5108 non-null   datetime64[ns]
 11  Claim ID              5108 non-null   object        
 12  G_Claim ID            5108 non-null   object        
 13  Scheme ID         

In [13]:
start_date = '2023-01-01'
end_date = '2023-12-31'
filtered_df = df[(df['Retail invoice Date'] >= start_date) & (df['Retail invoice Date'] <= end_date)]
print(filtered_df)

     Dealer Code                      Dealer Name   Dealer State Region  \
0        1007360  ASHOK AUTO VENTURES LLP-1007360  Uttar Pradesh  North   
1        1007360  ASHOK AUTO VENTURES LLP-1007360  Uttar Pradesh  North   
18       1009100            ASHOK AUTO SALES LTD.  Uttar Pradesh  North   
19       1007360  ASHOK AUTO VENTURES LLP-1007360  Uttar Pradesh  North   
20       1009100            ASHOK AUTO SALES LTD.  Uttar Pradesh  North   
...          ...                              ...            ...    ...   
4974     1007360  ASHOK AUTO VENTURES LLP-1007360  Uttar Pradesh  North   
4975     1007360  ASHOK AUTO VENTURES LLP-1007360  Uttar Pradesh  North   
4976     1007360  ASHOK AUTO VENTURES LLP-1007360  Uttar Pradesh  North   
4977     1007360  ASHOK AUTO VENTURES LLP-1007360  Uttar Pradesh  North   
5016     1007360  ASHOK AUTO VENTURES LLP-1007360  Uttar Pradesh  North   

       Status               Status Reson         Chassis no TMInvoiceno  \
0     Settled           

In [14]:
field_info = {
    'Dealer Code': 'The unique identifier for each dealer.',
    'Dealer Name': 'The name of the dealer associated with the claim.',
    'Dealer State': 'The state in which the dealer is located.',
    'Region': 'The geographical region of the dealer.',
    'Status': 'The current status of the claim (e.g., Settled, Pending).',
    'Status Reson': 'The reason for the current status of the claim.',
    'Chassis no': 'The unique chassis number of the vehicle associated with the claim.',
    'TMInvoiceno': 'The invoice number issued by the TM (Tata Motors) for the claim.',
    'TM Invoice Date': 'The date on which the TM invoice was issued.',
    'Retail invoice no': 'The invoice number issued by the retailer.',
    'Retail invoice Date': 'The date on which the retail invoice was issued.',
    'Claim ID': 'The unique identifier for the claim.',
    'G_Claim ID': 'The identifier for the group claim, if applicable.',
    'Scheme ID': 'The identifier for the scheme under which the claim is made.',
    'Amount': 'The monetary amount claimed.',
    'SAP Doc no.': 'The document number generated by SAP for the claim.',
    'SAP Doc Date': 'The date on which the SAP document was generated.',
    'Scheme Type': 'The type of scheme under which the claim is made.',
    'Scheme Type-Name': 'The name of the scheme under which the claim is made.',
    'Scheme Type Category': 'The category of the scheme under which the claim is made.',
    'LOB': 'The line of business (LOB) associated with the claim.'
}

In [15]:
connector = PandasConnector({"original_df": df}, field_descriptions=field_info)
sdf = SmartDataframe(connector,config={'llm':llm})

In [16]:
sdf.chat("Give me all the records for chasis no 'MAT558005PVJ50387'")

{'type': 'dataframe', 'value':   Dealer Code                      Dealer Name   Dealer State Region   Status  \
0     1007360  ASHOK AUTO VENTURES LLP-1007360  Uttar Pradesh  North  Settled   
1     1007360  ASHOK AUTO VENTURES LLP-1007360  Uttar Pradesh  North  Settled   

  Status Reson         Chassis no TMInvoiceno TM Invoice Date  \
0      On-Line  MAT558005PVJ50387   723276640      2023-09-29   
1      On-Line  MAT558005PVJ50387   723276640      2023-09-29   

  Retail invoice no  ...     Claim ID G_Claim ID     Scheme ID   Amount  \
0  IAUTGH2324001350  ...  CVA24848666          0  SCHM12-13493  10000.0   
1  IAUTGH2324001350  ...  CVB24880194          0  SCHM12-13480  10000.0   

   SAP Doc no. SAP Doc Date Scheme Type                 Scheme Type-Name  \
0    728322645   2024-04-06  Chola Cash  Cash discount in lieu of Chola    
1    728322646   2024-04-06    Exchange                Exchange Discount   

  Scheme Type Category      LOB  
0             Consumer  SCVPass  
1     

,Dealer Code,Dealer Name,Dealer State,Region,Status,Status Reson,Chassis no,TMInvoiceno,TM Invoice Date,Retail invoice no,...,Claim ID,G_Claim ID,Scheme ID,Amount,SAP Doc no.,SAP Doc Date,Scheme Type,Scheme Type-Name,Scheme Type Category,LOB
0,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT558005PVJ50387,723276640,2023-09-29,IAUTGH2324001350,...,CVA24848666,0,SCHM12-13493,10000.0,728322645,2024-04-06,Chola Cash,Cash discount in lieu of Chola,Consumer,SCVPass
1,1007360,ASHOK AUTO VENTURES LLP-1007360,Uttar Pradesh,North,Settled,On-Line,MAT558005PVJ50387,723276640,2023-09-29,IAUTGH2324001350,...,CVB24880194,0,SCHM12-13480,10000.0,728322646,2024-04-06,Exchange,Exchange Discount,Consumer,SCVPass


In [17]:
'MAT558005PVJ50387'

'MAT558005PVJ50387'

In [18]:
sdf.chat("what is total claim amount for lob 'Pickups' for retail invoices in a year 2021")

{'type': 'number', 'value': 24000.0}


24000.0

In [19]:


# # Sample DataFrame
# sales_by_country = pd.DataFrame({
#     "country": ["United States", "United Kingdom", "France", "Germany", "Italy", "Spain", "Canada", "Australia", "Japan", "China"],
#     "revenue": [5000, 3200, 2900, 4100, 2300, 2100, 2500, 2600, 4500, 7000]
# })

# # By default, unless you choose a different LLM, it will use BambooLLM.
# # You can get your free API key signing up at https://pandabi.ai (you can also configure it in your .env file)
# #os.environ["PANDASAI_API_KEY"] = "YOUR_API_KEY"

# agent = Agent(sales_by_country,config={'llm':llm})
# agent.chat('Which are the top 5 countries by sales?')


In [20]:
# agent = Agent(connector,config={'llm':llm})
# agent.chat('how mant records are there?')